In [1]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_analyzer import CustomPairPlot
import graphviz
import pydotplus
from IPython.display import Image
from IPython.display import HTML
from six import StringIO
from ipywidgets import interact, FloatSlider

# 前処理
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# データセット分割
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 特徴量選択
from sklearn.feature_selection import (
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)
from boruta import BorutaPy

# 学習中
import optuna
from tqdm import tqdm
from sklearn.model_selection import learning_curve, cross_validate, cross_val_score

# 評価指標
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings


# config python file
import config

SEED = config.SEED


from functions import *

fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

/root/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/breast_cancer_analytics/notebooks/functions.py:278: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y: pd.Series(),


# 目的
遺伝子学的分類に基づいた、予後の2値分類を実施する。  
分類はCLAUDIN_SUBTYPEに基づいて実施。  
予後は5年、10年、15年の3つの年次に分けている。Trueで死亡であることに注意すること。

# データ読み込み
読み込み元：
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/claudin_subtype_chi2"

サブタイプ毎のデータを使用

データの種類が多いので、辞書型で表現する  

In [2]:
# ディレクトリ構造を辞書に反映するための関数
def dir2dict(dic, path):
    for k in os.listdir(path):
        if os.path.isdir(os.path.join(path, k)):
            if not k in dic:
                dic[k] = dict()
            dir2dict(dic[k], path + "/" + k)
        else:
            if k[0] == "X" or k[0] == "y":
                dic[k.split(".")[0]] = pd.read_pickle(path + "/" + k)

In [3]:
df_dict = dict()
dir2dict(df_dict, config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR)

# モデルのトレーニング

## データ全体のベースライン・学習

### boruta適用データのベースライン・基本学習結果

In [4]:
for year in range(5, 16, 5):  # 予後年数毎のループ

    X_train_tmp = df_dict["chi2"]["boruta"]["train"]["X{0:0=2}".format(year)]
    y_train_tmp = df_dict["chi2"]["boruta"]["train"]["y{0:0=2}".format(year)]
    X_test_tmp = df_dict["chi2"]["boruta"]["test"]["X{0:0=2}".format(year)]
    y_test_tmp = df_dict["chi2"]["boruta"]["test"]["y{0:0=2}".format(year)]
    assert X_train_tmp.shape[0] == y_train_tmp.shape[0], "train size is incorrect"
    assert X_test_tmp.shape[0] == y_test_tmp.shape[0], "test size is incorrect"

    # accuracyの表示
    print("----------" * 10)
    print("予後年数：{0:0=2}年:".format(year))
    if accuracy_score(y_train_tmp, np.zeros(len(y_train_tmp))) >= 0.5:
        score = (
            "0>1".format(year),
            round(accuracy_score(y_train_tmp, np.zeros(len(y_train_tmp))), 3),
        )
    else:
        score = (
            "0>1".format(year),
            round(accuracy_score(y_train_tmp, np.ones(len(y_train_tmp))), 3),
        )
    print("accuracyベースライン：", score)
    print("使用特徴量：", X_train_tmp.columns)
    print("学習サンプルサイズ：", X_train_tmp.shape)
    display("ラベル比率：", y_train_tmp.value_counts())
    display(compare_bcms(X_train_tmp, y_train_tmp))

----------------------------------------------------------------------------------------------------
予後年数：05年:
accuracyベースライン： ('0>1', 0.812)
使用特徴量： Index(['BCL2', 'C1orf106', 'C6orf97', 'CDCA5', 'ESR1', 'EXO1', 'FAM83D',
       'FGD3', 'FGFR4', 'HPN', 'IL6ST', 'KIF20A', 'KRT80', 'MAPT', 'PREX1',
       'SERPINA3', 'SUSD3', 'TMEM26'],
      dtype='object')
学習サンプルサイズ： (1306, 18)


'ラベル比率：'

0    1060
1     246
Name: OS_05years, dtype: int64

11it [12:16, 66.92s/it]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Naive Bayes,0.722308,0.721321,0.462233,0.457753
Quadratic Discriminant Analysis,0.783733,0.739630,0.468855,0.343321
AdaBoost,0.857410,0.799389,0.501189,0.314517
Decision Tree,0.862344,0.777962,0.560306,0.298459
Nearest Neighbors,0.841927,0.778667,0.439279,0.224470
Polynomial SVM,0.840564,0.805514,0.330344,0.162169
Logistic Regression,0.816403,0.811615,0.171174,0.128121
Random Forest,0.844309,0.810869,0.312603,0.118968
Linear SVM,0.811639,0.811650,0.000000,0.000000


...

----------------------------------------------------------------------------------------------------
予後年数：15年:
accuracyベースライン： ('0>1', 0.533)
使用特徴量： Index(['AURKA', 'CCL19', 'CIDEC', 'CLEC3A', 'CLIC6', 'CYP4F22', 'DARC', 'FGD3',
       'HIST1H4H', 'LOC389033', 'MAPT', 'MFAP4', 'MYBPC1', 'NAT1', 'PLIN4',
       'S100P', 'SERPINA3', 'SFRP1', 'SPP1', 'SUSD3', 'TAT', 'TMEM26', 'UBE2C',
       'VTCN1'],
      dtype='object')
学習サンプルサイズ： (811, 24)


'ラベル比率：'

1    432
0    379
Name: OS_15years, dtype: int64

11it [03:37, 19.77s/it]


,acc_train,acc_test,f1_train,f1_test
classifier,,,,
Random Forest,0.807370,0.653538,0.829001,0.696584
Sigmoid SVM,0.532677,0.532746,0.695061,0.692588
RBF SVM,0.735717,0.638783,0.773667,0.690699
Logistic Regression,0.674475,0.652288,0.708195,0.682886
Naive Bayes,0.645980,0.641238,0.684000,0.676557
Linear SVM,0.677215,0.632520,0.712804,0.668490
Nearest Neighbors,0.752706,0.614107,0.781027,0.659791
AdaBoost,0.798192,0.625173,0.813878,0.657457
Quadratic Discriminant Analysis,0.734622,0.611593,0.766278,0.654535


## subtype毎のベースライン・学習

### borutaを使用した場合のベースライン・基本学習結果

In [5]:
pd.read_pickle(
    config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/df_cross.pkl"
)["CLAUDIN_SUBTYPE"].unique()

array(['claudin-low', 'LumA', 'LumB', 'Her2', 'Normal', 'Basal'],
      dtype=object)

for year in range(5, 16, 5):  # 予後年数毎のループ
    print("====={0:0=2}".format(year) * 10)

    for subtype in pd.read_pickle(
        config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/df_cross.pkl"
    )["CLAUDIN_SUBTYPE"].unique():
        X_train_tmp = df_dict["chi2"]["claudin_subtype"]["train"][
            "X{0:0=2}_{1}".format(year, subtype)
        ]
        y_train_tmp = df_dict["chi2"]["claudin_subtype"]["train"][
            "y{0:0=2}_{1}".format(year, subtype)
        ]
        X_test_tmp = df_dict["chi2"]["claudin_subtype"]["test"][
            "X{0:0=2}_{1}".format(year, subtype)
        ]
        y_test_tmp = df_dict["chi2"]["claudin_subtype"]["test"][
            "y{0:0=2}_{1}".format(year, subtype)
        ]
        assert X_train_tmp.shape[0] == y_train_tmp.shape[0], "train size is incorrect"
        assert X_test_tmp.shape[0] == y_test_tmp.shape[0], "test size is incorrect"

        # accuracyの表示
        print("----------" * 10)
        print("subtype: ", subtype)
        print("予後年数：{0:0=2}年:".format(year))
        if accuracy_score(y_train_tmp, np.zeros(len(y_train_tmp))) >= 0.5:
            score = (
                "0>1".format(year),
                round(accuracy_score(y_train_tmp, np.zeros(len(y_train_tmp))), 3),
            )
        else:
            score = (
                "0>1".format(year),
                round(accuracy_score(y_train_tmp, np.ones(len(y_train_tmp))), 3),
            )
        print("accuracyベースライン：", score)
        print("使用特徴量：", X_train_tmp.columns)
        print("学習サンプルサイズ：", X_train_tmp.shape)
        display("ラベル比率：", y_train_tmp.value_counts())
        display(compare_bcms(X_train_tmp, y_train_tmp))

# 予測・最適化

分類器を学習させ、パラメータのチューニングを行い、高い予測精度を目指す。

## optuna

モデルのパラメータをベイズ最適化に基づいて最適化していくoptunaを使用する

### Random Forest


In [7]:
def objective(trial):
    # ランダムフォレストのパラメータチューニング
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
    max_depth = trial.suggest_int("max_depth", 2, 50, log=True)
    max_leaf_noddes = trial.suggest_int("max_leaf_nodes", 2, 100)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_noddes,
        max_features=max_features,
        random_state=SEED,
    )
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X, y, n_jobs=-1, cv=10, scoring=make_scorer(f1_score))
    accuracy = score.mean()
    return accuracy

##　全サンプルでの予測(boruta)

In [8]:
year = 15
X_train_tmp = df_dict["chi2"]["boruta"]["train"]["X{0:0=2}".format(year)]
y_train_tmp = df_dict["chi2"]["boruta"]["train"]["y{0:0=2}".format(year)]
X_test_tmp = df_dict["chi2"]["boruta"]["test"]["X{0:0=2}".format(year)]
y_test_tmp = df_dict["chi2"]["boruta"]["test"]["y{0:0=2}".format(year)]

X, y = X_train_tmp.copy(), y_train_tmp.copy()
study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
)
study.optimize(objective, n_trials=100)

[I 2022-08-11 09:32:42,633] A new study created in memory with name: no-name-31359920-1ec0-42be-b0e7-645d3bc302b3
/root/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/root/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/root/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/root/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is 

In [9]:
# 最も良いパラメータ
print(f"The best value is : \n {study.best_value}")
print(f"The best parameters are : \n {study.best_params}")

The best value is : 
 0.7177175017522909
The best parameters are : 
 {'n_estimators': 945, 'criterion': 'gini', 'max_depth': 3, 'max_leaf_nodes': 44, 'max_features': 'log2'}


In [10]:
print("tuning前")
rf = RandomForestClassifier(random_state=SEED)
rf.fit(X_train_tmp, y_train_tmp)
pred_tmp = rf.predict(X_test_tmp)
show_scores(y_test_tmp, pred_tmp)

print("tuning後")
rf = RandomForestClassifier(
    n_estimators=705,
    criterion="entropy",
    max_depth=2,
    max_leaf_nodes=13,
    max_features="log2",
    random_state=SEED,
)
rf.fit(X_train_tmp, y_train_tmp)
pred_tmp = rf.predict(X_test_tmp)
show_scores(y_test_tmp, pred_tmp)

tuning前
accuracy:  0.6201550387596899
precision:  0.625
recall:  0.7246376811594203
f1 score:  0.6711409395973154
tuning後
accuracy:  0.6782945736434108
precision:  0.655367231638418
recall:  0.8405797101449275
f1 score:  0.7365079365079366


In [16]:
# optunaの過程を可視化
optuna.visualization.plot_optimization_history(study).show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

## subtype毎の予測

In [17]:
for year in range(5, 16, 5):  # 予後年数毎のループ
    print("====={0:0=2}".format(year) * 10)

    for subtype in pd.read_pickle(
        config.INTERIM_PICKLE_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/df_cross.pkl"
    )["CLAUDIN_SUBTYPE"].unique():
        print("-----" * 10)
        print(subtype)
        X_train_tmp = df_dict["chi2"]["claudin_subtype"]["train"][
            "X{0:0=2}_{1}".format(year, subtype)
        ]
        y_train_tmp = df_dict["chi2"]["claudin_subtype"]["train"][
            "y{0:0=2}_{1}".format(year, subtype)
        ]
        X_test_tmp = df_dict["chi2"]["claudin_subtype"]["test"][
            "X{0:0=2}_{1}".format(year, subtype)
        ]
        y_test_tmp = df_dict["chi2"]["claudin_subtype"]["test"][
            "y{0:0=2}_{1}".format(year, subtype)
        ]
        assert X_train_tmp.shape[0] == y_train_tmp.shape[0], "train size is incorrect"
        assert X_test_tmp.shape[0] == y_test_tmp.shape[0], "test size is incorrect"

        print("tuning前")
        rf = RandomForestClassifier(random_state=SEED)
        rf.fit(X_train_tmp, y_train_tmp)
        pred_tmp = rf.predict(X_test_tmp)
        show_scores(y_test_tmp, pred_tmp)

        print("tuning後")
        rf = RandomForestClassifier(
            n_estimators=705,
            criterion="entropy",
            max_depth=2,
            max_leaf_nodes=13,
            max_features="log2",
            random_state=SEED,
        )
        rf.fit(X_train_tmp, y_train_tmp)
        pred_tmp = rf.predict(X_test_tmp)
        show_scores(y_test_tmp, pred_tmp)

=====05=====05=====05=====05=====05=====05=====05=====05=====05=====05
--------------------------------------------------
claudin-low
tuning前
accuracy:  0.7954545454545454
precision:  0.0
recall:  0.0
f1 score:  0.0
tuning後
accuracy:  0.8181818181818182
precision:  0.0
recall:  0.0
f1 score:  0.0
--------------------------------------------------
LumA
tuning前


/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:  0.9261744966442953
precision:  0.3333333333333333
recall:  0.1
f1 score:  0.15384615384615383
tuning後
accuracy:  0.9328859060402684
precision:  0.0
recall:  0.0
f1 score:  0.0
--------------------------------------------------
LumB
tuning前


/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:  0.8260869565217391
precision:  0.3333333333333333
recall:  0.05263157894736842
f1 score:  0.09090909090909091
tuning後
accuracy:  0.8347826086956521
precision:  0.0
recall:  0.0
f1 score:  0.0
--------------------------------------------------
Her2
tuning前


/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:  0.7058823529411765
precision:  0.6666666666666666
recall:  0.3333333333333333
f1 score:  0.4444444444444444
tuning後
accuracy:  0.6470588235294118
precision:  0.5
recall:  0.16666666666666666
f1 score:  0.25
--------------------------------------------------
Normal
tuning前
accuracy:  0.7714285714285715
precision:  0.25
recall:  0.5
f1 score:  0.3333333333333333
tuning後
accuracy:  0.7428571428571429
precision:  0.14285714285714285
recall:  0.25
f1 score:  0.18181818181818182
--------------------------------------------------
Basal
tuning前
accuracy:  0.4523809523809524
precision:  0.3
recall:  0.15789473684210525
f1 score:  0.20689655172413793
tuning後
accuracy:  0.47619047619047616
precision:  0.2857142857142857
recall:  0.10526315789473684
f1 score:  0.15384615384615385
=====10=====10=====10=====10=====10=====10=====10=====10=====10=====10
--------------------------------------------------
claudin-low
tuning前
accuracy:  0.6285714285714286
precision:  0.42857142857142855
recall

/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:  0.5268817204301075
precision:  0.3142857142857143
recall:  0.3548387096774194
f1 score:  0.3333333333333333
tuning後
accuracy:  0.5913978494623656
precision:  0.36
recall:  0.2903225806451613
f1 score:  0.3214285714285714
--------------------------------------------------
Her2
tuning前
accuracy:  0.574468085106383
precision:  0.5517241379310345
recall:  0.6956521739130435
f1 score:  0.6153846153846154
tuning後
accuracy:  0.5319148936170213
precision:  0.5142857142857142
recall:  0.782608695652174
f1 score:  0.6206896551724138
--------------------------------------------------
Normal
tuning前
accuracy:  0.6666666666666666
precision:  0.45454545454545453
recall:  0.7142857142857143
f1 score:  0.5555555555555556
tuning後
accuracy:  0.7083333333333334
precision:  0.5
recall:  0.7142857142857143
f1 score:  0.588235294117647
--------------------------------------------------
Basal
tuning前
accuracy:  0.5263157894736842
precision:  0.6923076923076923
recall:  0.391304347826087
f1 score: 

### lightGBM

In [ ]:
from lightgbm import LGBMClassifier


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
    }
    clf = LGBMClassifier(boosting_type="gbdt", **params, random_state=SEED)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X, y, n_jobs=-1, cv=10)
    accuracy = score.mean()
    return accuracy


year = 15
X_train_tmp = df_dict["chi2"]["boruta"]["train"]["X{0:0=2}".format(year)]
y_train_tmp = df_dict["chi2"]["boruta"]["train"]["y{0:0=2}".format(year)]
X_test_tmp = df_dict["chi2"]["boruta"]["test"]["X{0:0=2}".format(year)]
y_test_tmp = df_dict["chi2"]["boruta"]["test"]["y{0:0=2}".format(year)]

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

In [ ]:
# 最も良いパラメータ
print(f"The best value is : \n {study.best_value}")
print(f"The best parameters are : \n {study.best_params}")